In [2]:
!activate ds


(ds) C:\Users\Christian\Desktop\Nasa\Model_1_short_segmentation\Training_code\code\src\sample\submission>conda.bat activate ds 


In [114]:
import sys

import numpy as np 
import sys,os
import cv2
import matplotlib.pyplot as plt
import argparse
import pandas as pd
from utils import *
import random
from operator import itemgetter

In [201]:
wd = os.getcwd()
print("working directory is ", wd)

working directory is  C:\Users\Administrador\Documents\QS\Model_1_short_segmentation\Training_code\code\train


In [194]:
os.chdir("..")

In [195]:
os.chdir("..")

In [196]:
os.chdir("..")

In [198]:
os.chdir("./train")

In [202]:
wd = os.getcwd()
print("working directory is ", wd)

working directory is  C:\Users\Administrador\Documents\QS\Model_1_short_segmentation\Training_code\code\train


In [203]:
os.listdir('.')

['images', 'train_data.csv', 'videos', 'videos-frames']

In [143]:
#root_dir = "C:\\Users\\Administrador\\Documents\\QS\\NASA_SpatiumAESS\\Training_code\\code\\data\\train"

In [204]:
root_dir

'C:\\Users\\Administrador\\Documents\\QS\\NASA_SpatiumAESS\\Training_code\\code\\data\\train'

In [205]:
# Load data paths
training_images = []   # images png + jpg
img_ann = []        # images annotations
    
videos = []         # videos
videos_ann = []     # videos annotations
video_frames = []   # video frames
video_frames_ann=[] # video frames annotations

for root, dirs, files in os.walk(root_dir, topdown=True):
    for name in files:
        path = os.path.join(root, name)            
        if '.png' in path  and 'images' in path:
            training_images.append(path)
            print(path)
        if '.jpg' in path  and 'images' in path:
            training_images.append(path)
        if '.txt' in path and 'images' in path:
            img_ann.append(path)
        if '.mov' in path and 'videos' in path:
            videos.append(path)
        if '.csv' in path and 'videos' in path:
            videos_ann.append(path)
        if '.jpg' in path and 'videos-frames' in path:
            video_frames.append(path)
            #if '.csv' in path and 'videos-frames' in path: #TODO: CHECK ANNOTATIONS
            #    video_frames_ann.append(path)
print('Image training instances: ', len(training_images))
print('videos training instances: ', len(videos))
print('video_frames evaluation instances: ', len(video_frames))


C:\Users\Administrador\Documents\QS\NASA_SpatiumAESS\Training_code\code\data\train\images\047174.png
C:\Users\Administrador\Documents\QS\NASA_SpatiumAESS\Training_code\code\data\train\images\049349.png
C:\Users\Administrador\Documents\QS\NASA_SpatiumAESS\Training_code\code\data\train\images\078630.png
C:\Users\Administrador\Documents\QS\NASA_SpatiumAESS\Training_code\code\data\train\images\084510.png
C:\Users\Administrador\Documents\QS\NASA_SpatiumAESS\Training_code\code\data\train\images\085302.png
C:\Users\Administrador\Documents\QS\NASA_SpatiumAESS\Training_code\code\data\train\images\091632.png
C:\Users\Administrador\Documents\QS\NASA_SpatiumAESS\Training_code\code\data\train\images\105779.png
C:\Users\Administrador\Documents\QS\NASA_SpatiumAESS\Training_code\code\data\train\images\118330.png
C:\Users\Administrador\Documents\QS\NASA_SpatiumAESS\Training_code\code\data\train\images\134928.png
C:\Users\Administrador\Documents\QS\NASA_SpatiumAESS\Training_code\code\data\train\images\1

In [10]:
video_frames

['C:\\Users\\Administrador\\Documents\\QS\\NASA_SpatiumAESS\\Training_code\\code\\data\\train\\videos-frames\\EMU-04-0001.jpg',
 'C:\\Users\\Administrador\\Documents\\QS\\NASA_SpatiumAESS\\Training_code\\code\\data\\train\\videos-frames\\EMU-04-0002.jpg',
 'C:\\Users\\Administrador\\Documents\\QS\\NASA_SpatiumAESS\\Training_code\\code\\data\\train\\videos-frames\\EMU-04-0003.jpg',
 'C:\\Users\\Administrador\\Documents\\QS\\NASA_SpatiumAESS\\Training_code\\code\\data\\train\\videos-frames\\EMU-04-0004.jpg',
 'C:\\Users\\Administrador\\Documents\\QS\\NASA_SpatiumAESS\\Training_code\\code\\data\\train\\videos-frames\\EMU-04-0005.jpg',
 'C:\\Users\\Administrador\\Documents\\QS\\NASA_SpatiumAESS\\Training_code\\code\\data\\train\\videos-frames\\EMU-04-0006.jpg',
 'C:\\Users\\Administrador\\Documents\\QS\\NASA_SpatiumAESS\\Training_code\\code\\data\\train\\videos-frames\\EMU-04-0007.jpg',
 'C:\\Users\\Administrador\\Documents\\QS\\NASA_SpatiumAESS\\Training_code\\code\\data\\train\\videos-fr

# Image segmentation  

In [11]:

def segmentation(path, label):
    
    flag = False 

    IMG_SIZE = 500
    
    image = cv2.imread(path)

    mask = np.zeros(image.shape[:2], np.uint8) # create a similar mask image

    backgroundModel = np.zeros((1, 65), np.float64) # These are arrays used by the algorithm internally. You just create two np.float64 type zero arrays of size (1,65).
    
    foregroundModel = np.zeros((1, 65), np.float64) # These are arrays used by the algorithm internally. You just create two np.float64 type zero arrays of size (1,65).

    rectangle = (50,50,450,290)

    cv2.grabCut(image, mask, rectangle,  
                backgroundModel, foregroundModel,
                3, cv2.GC_INIT_WITH_RECT)

    mask2 = np.where((mask==2)|(mask==0),0,1).astype('uint8') # apply mask
    # So we modify the mask such that all 0-pixels and 2-pixels are put to 0 (ie background) and all 1-pixels and 3-pixels are put to 1(ie foreground pixels). Now our final mask is ready. Just multiply it with input image to get the segmented image. 
    #print(mask2)
    img = image*mask2[:,:,np.newaxis]
    
    if (np.mean(mask2)!= 0):
        img = image*mask2[:,:,np.newaxis]

        [rows,cols]= mask2.shape

        triplets = []
        for i in range(rows):
            for j in range(cols):
                k = mask2[i,j]
                if k == 1:
                    aux = [i,j,int(random.uniform(0,3))]

                    triplets.append(aux)
                    #print(triplets)
                    #mask[i,j] = 1
                else: # ADD HERE LABEL 1 AND LABEL 0
                    aux = 0
                    #print("no")
                    #mask[i,j] = 0

        #-----------------Select 15 TRIPLETS FROM SEGMENTED IMAGE----------------------------
        points = []
        # CALCULATE THE 15 TRIPLETS COORDINATES THAT HAVE THE MAX DIST WITH dist = numpy.linalg.norm(a-b)
        # Body-part detection 
        for i in range(15):
            points.append(int(random.triangular(0,len(triplets)))) # len(triplets) bc of all possible triplets on segmentated img

        #-----------------15 TRIPLETS SELECTION-------------------------------------
        sel  = itemgetter(*points)(triplets)
        #print("Selected 15 triplets: ", len(sel))
        #print(sel)

        #----------------SELECT AREAS---------------------------------------------------------
        #print("x ", min(sel))
        triplet_min = min(sel)
        #print("y ", max(sel))
        triplet_max = max(sel)
        x_i,y_i,l_i = triplet_min
        x_f,y_f,l_f = triplet_max

        c1 = x_i,y_i
        c2 = x_f,y_i
        c3 = x_f,y_f
        c4 = x_i,y_f

        rect = c1 + c2 + c3 + c4 + c1 # 5 pairs 
        #print("Polygon coordenates ",rect)

        #plt.imshow(img)
        #plt.colorbar()
        #plt.show()
        #print(final)
        flag = False # Keep false of there is detection
        #print('{} - {}'.format(sel,rect))
    else:
        print("no obj detected")
        sel = []
        rect = []
        flag = True # turn to truth if there is not detection
        #print('{} - {}'.format(sel,rect))
    
    #plt.imshow(mask2)
    
    return sel, rect, flag
# Fast testing bef assignmt



In [12]:
def feature_extraction(path, label): # ADAPT ANNOTATIONS CORRECTLY & POLYGON + AREA EXTRACTION
  #ima = cv2.imread(path, cv2.IMREAD_GRAYSCALE)
  
  triplets, coordinates,flag = segmentation(path, label)
  
  triplets = np.array( [triplets]  ).ravel().ravel()  #+ coordinates
  triplets = list(triplets)
  return triplets, coordinates,flag

# # Fast testing bef assignmt
# path = "/home/sebasmos/Documentos/NASA_Spacesuit/test/images/539372.png"

# triplets, coordinates,flag = feature_extraction(path, 1)
# coordinates

In [206]:
i = 0
# Training with images 
Num_features = 1
feat_names = ['Polygon'] # TODO: EXTRACT TRIPLETS HERE!
train_size = len(training_images)
final = []

data_train = []# np.zeros((train_size, Num_features+1), dtype=np.float32)
#data_test = np.zeros((test_size, Num_features+1), dtype=np.float32)

#--------------------------IMAGES-----------------------------------------------------------
print("Creating solutions/images/annotations/solutions.csv...")
for path in training_images:
    print('    {}/{}...'.format(i+1, len(training_images)))
    
    triplets, coordinates,flag = feature_extraction(path, 2)
    
    id_img = path[len(path) - 10:len(path)]
    
    if flag==True:
        row = id_img
    else:
        row = id_img + ',' +str(triplets)[1:-1] + ','+ '[' + str(coordinates) + ']' # working with lists
    print(' {} '.format(row))
    
    #data_train.append(feature_extraction(path, 2))
    # Extract id_name from image

    # Add id + triplets + geoms
    row_str = " ".join(map(str, row))
    file1 = open("solutions_final.csv","a")
    file1.write(row + " \n")
    file1.close()
    i = i+1

Creating solutions/images/annotations/solutions.csv...
    1/86...
 020267.jpg,199, 327, 2, 230, 190, 0, 215, 221, 1, 190, 366, 0, 228, 394, 1, 212, 395, 0, 285, 370, 0, 301, 416, 0, 272, 340, 1, 284, 466, 0, 229, 292, 2, 109, 296, 1, 187, 263, 0, 151, 247, 1, 266, 194, 0,[(109, 296, 301, 296, 301, 416, 109, 416, 109, 296)] 
    2/86...


KeyboardInterrupt: 

# Video segmentation  - different format

In [14]:

def video_segmentation(path, label):
    
    flag = False 

    IMG_SIZE = 500
    
    image = cv2.imread(path)

    mask = np.zeros(image.shape[:2], np.uint8) # create a similar mask image

    backgroundModel = np.zeros((1, 65), np.float64) # These are arrays used by the algorithm internally. You just create two np.float64 type zero arrays of size (1,65).
    
    foregroundModel = np.zeros((1, 65), np.float64) # These are arrays used by the algorithm internally. You just create two np.float64 type zero arrays of size (1,65).

    rectangle = (50,50,450,290)

    cv2.grabCut(image, mask, rectangle,  
                backgroundModel, foregroundModel,
                3, cv2.GC_INIT_WITH_RECT)

    mask2 = np.where((mask==2)|(mask==0),0,1).astype('uint8') # apply mask
    # So we modify the mask such that all 0-pixels and 2-pixels are put to 0 (ie background) and all 1-pixels and 3-pixels are put to 1(ie foreground pixels). Now our final mask is ready. Just multiply it with input image to get the segmented image. 
    #print(mask2)
    img = image*mask2[:,:,np.newaxis]
    
    if (np.mean(mask2)!= 0):
        img = image*mask2[:,:,np.newaxis]

        [rows,cols]= mask2.shape

        triplets = []
        for i in range(rows):
            for j in range(cols):
                k = mask2[i,j]
                if k == 1:
                    aux = [i,j,int(random.uniform(0,cols))]

                    triplets.append(aux)
                    #print(triplets)
                    #mask[i,j] = 1
                else: # ADD HERE LABEL 1 AND LABEL 0
                    aux = 0
                    #print("no")
                    #mask[i,j] = 0

        #-----------------Select 15 TRIPLETS FROM SEGMENTED IMAGE----------------------------
        points = []
        # CALCULATE THE 15 TRIPLETS COORDINATES THAT HAVE THE MAX DIST WITH dist = numpy.linalg.norm(a-b)
        # Body-part detection 
        for i in range(13):
            points.append(int(random.triangular(0,len(triplets)))) # len(triplets) bc of all possible triplets on segmentated img

        #-----------------15 TRIPLETS SELECTION-------------------------------------
        sel  = itemgetter(*points)(triplets)
        #print("Selected 15 triplets: ", len(sel))
        #print(sel)

        #----------------SELECT AREAS---------------------------------------------------------
        #print("x ", min(sel))
        triplet_min = min(sel)
        #print("y ", max(sel))
        triplet_max = max(sel)
        x_i,y_i,l_i = triplet_min
        x_f,y_f,l_f = triplet_max

        c1 = x_i,y_i
        c2 = x_f,y_i
        c3 = x_f,y_f
        c4 = x_i,y_f

        rect = c1 + c2 + c3 + c4 + c1 # 5 pairs 
        #print("Polygon coordenates ",rect)

        #plt.imshow(img)
        #plt.colorbar()
        #plt.show()
        #print(final)
        flag = False # Keep false of there is detection
        #print('{} - {}'.format(sel,rect))
    else:
        print("no obj detected")
        sel = []
        rect = []
        flag = True # turn to truth if there is not detection
        #print('{} - {}'.format(sel,rect))
    
    #plt.imshow(mask2)
    
    
    return sel, flag

def video_feature_extraction(path, label): # ADAPT ANNOTATIONS CORRECTLY & POLYGON + AREA EXTRACTION
  #ima = cv2.imread(path, cv2.IMREAD_GRAYSCALE)
  triplets, flag = video_segmentation(path, label)
  
  triplets = np.array( [triplets]  ).ravel().ravel()  #+ coordinates
  triplets = list(triplets)
  return triplets, flag

# Fast testing bef assignmt
# path = "/home/sebasmos/Documentos/NASA_Spacesuit/train/videos-frames/MKIII-06-0111.jpg"
# triplets, coordinates,flag = video_feature_extraction(path, 1)

# EXTRACT anotations solutions

In [16]:
video_frames

def extractVideoNames(framesList):
    videos = [] 
    
    for path in framesList:
        name = os.path.basename(path)#path[len(path)-15:-1]
        name = name[:len(name)-9]
        videos.append(name)
    return list(set(videos))
videos = extractVideoNames(video_frames)
videos

['MKIII-24',
 'MKIII-04',
 'MKIII-03',
 'MKIII-16',
 'MKIII-12',
 'MKII_08',
 'EMU-10',
 'MKIII-06',
 'EMU-04',
 'EMU-05',
 'EMU-12',
 'MKIII-21',
 'EMU-09']

In [56]:
topheader = 'Frame #,CLAV,,,RSJC,,,LSJC,,,REJC,,,LEJC,,,RWJC,,,LWJC,,,RHJC,,,LHJC,,,RKJC,,,LKJC,,,RAJC,,,LAJC,,'
header = ['']+['X','Y','Z']*13
header

['',
 'X',
 'Y',
 'Z',
 'X',
 'Y',
 'Z',
 'X',
 'Y',
 'Z',
 'X',
 'Y',
 'Z',
 'X',
 'Y',
 'Z',
 'X',
 'Y',
 'Z',
 'X',
 'Y',
 'Z',
 'X',
 'Y',
 'Z',
 'X',
 'Y',
 'Z',
 'X',
 'Y',
 'Z',
 'X',
 'Y',
 'Z',
 'X',
 'Y',
 'Z',
 'X',
 'Y',
 'Z']

In [62]:
import os

videos_indexes_dict = {vid:1 for vid in videos}

for frame_path in video_frames:
    
    for v in videos:
        if v in frame_path:
        
            path = frame_path
            header_str = ",".join(header)+"\n"
            file1 = open(path_,"a")
            if videos_indexes_dict[v]==1:
                print(topheader + " \n" + header_str)
                file1.write(topheader + " \n" + header_str)
            triplets, flag = video_feature_extraction(path , 2)
            
            id_img = frame_path[len(path) - 15:len(path)]
#             id_img=''
#             if flag==True:
#                 row = id_img
#             else:
#                 row = id_img + ',' +str(triplets)[1:-1]
            row = str(triplets)[1:-1]
            print(' {} '.format(row))
            
            #record the first column index
            row = str(videos_indexes_dict[v])+ "," + row
            
            
            anotations_path = "{}{}{}{}{}".format(root_dir,os.sep,'videos',os.sep,"annotations")
            
            # create anotations folder if not exists
            
            try:
                os.makedirs(anotations_path)
            except: 
                pass
            
            
            path_ = "{}{}{}".format(anotations_path,os.sep,v+".csv")
            
            row_str = " ".join(map(str, row))
            
            
            
            file1.write(row + " \n")
            file1.close()
            videos_indexes_dict[v]+=1

Frame #,CLAV,,,RSJC,,,LSJC,,,REJC,,,LEJC,,,RWJC,,,LWJC,,,RHJC,,,LHJC,,,RKJC,,,LKJC,,,RAJC,,,LAJC,, 
,X,Y,Z,X,Y,Z,X,Y,Z,X,Y,Z,X,Y,Z,X,Y,Z,X,Y,Z,X,Y,Z,X,Y,Z,X,Y,Z,X,Y,Z,X,Y,Z,X,Y,Z

 258, 137, 239, 261, 141, 599, 235, 445, 219, 126, 375, 138, 258, 383, 516, 275, 261, 523, 168, 266, 627, 204, 179, 600, 179, 335, 198, 222, 440, 372, 111, 344, 381, 188, 430, 144, 258, 344, 422 
 252, 413, 600, 318, 323, 178, 147, 310, 15, 173, 153, 234, 189, 489, 461, 172, 476, 101, 298, 158, 288, 130, 282, 300, 208, 350, 484, 208, 410, 603, 258, 402, 602, 157, 223, 440, 189, 161, 548 
 112, 423, 201, 127, 188, 421, 252, 285, 144, 284, 172, 291, 224, 374, 400, 124, 266, 618, 204, 193, 269, 217, 470, 244, 245, 232, 400, 211, 460, 377, 225, 149, 141, 223, 339, 593, 183, 226, 145 
 223, 219, 203, 253, 444, 213, 184, 490, 262, 162, 309, 608, 134, 355, 531, 208, 301, 542, 201, 216, 228, 229, 341, 97, 272, 453, 619, 191, 462, 467, 216, 391, 575, 236, 449, 391, 153, 338, 626 
 128, 461, 210, 224, 237, 456, 220, 23

KeyboardInterrupt: 

Store Features (joint coordinates, areas, polygons) to csv

train_size = len(training_images)
print("training data: ", train_size)
train_pd = pd.read_csv("/home/sebasmos/Documentos/NASA_Spacesuit/NASA_SpatiumAESS/Training_code/code/src/sample/submission/training_images.csv")

Num_suits = 1 # TO UPDATE ACCORDING TO IMAGE
Triplets = 15 #
aux = Num_suits*Triplets
#joints = np.zeros((train_size, Num_suits*Triplets))

#train ? 
id_images = []
aux = len(path) - 10
for path in training_images:
    id_images.append(path[aux:len(path)])


def gen_array():
    x_f = []
    y_f = []
    v_f = []

    for i in range(15):
        x_f.append(int (random.uniform(100,999)))
        y_f.append(int (random.uniform(100,999)))
        v_f.append(int (random.uniform(0,3)))

    return x_f,y_f,v_f


def gen_geometries():
    x_f = []
    y_f = []
    type = 0
    if type == 0: #rectangular then 8 var, 4 corners
        for i in range(4):
            x_f.append(int (random.uniform(100,999)))
            y_f.append(int (random.uniform(100,999)))
    elif type==1: # 3 corners
        print("triangular")
    else:
        print("no figure detected")
    return x_f,y_f

def generar_csv (imag,coo,polig):

   f=open('../../../../solution/solution.csv','w')

   for l in range (len(imag)):
      
      f.write(str(imag[l])+','+str(coo[l])+','+str(polig[l])+'\n')

   f.close()

joints_id = []
geom = []

for id, j in enumerate(training_images):

    for i in range(Num_suits):
            #print(" es multiplo de 3")
            x_f,y_f,v_f = gen_array()
            joints_id.append(x_f + y_f + v_f)
            x_geom, y_geom = gen_geometries()
            geom.append(x_geom + y_geom)
            
print("id_images: ",train_size)
print("joints_id: ",len(joints_id))
print("geom: ",len(geom))

generar_csv (id_images,joints_id,geom)



